In [29]:
#############################
### Import libraries ###
#############################

from docplex.cp.model import *
from docplex.cp.config import get_default
import numpy as np
from Solver import *
from User import *
import time

In [30]:
#############################
### Essentials functions ###
#############################
import FunctionMain as fm

In [31]:
# ----------------- Parameters

# The file to consider
file = './file_with_optimal_val/la04.txt'

# The number of solutions that we will have in the solver during the first iteration
k = 10

# The number of layer which is fixed
nb_layers = 5

# Variable which display the solution
display_sol = False

# Variable which display the start (in a vector)
display_start = False

# Variable which display the start (in a matrix)
display_matrix = False

# Time stop criterion 
tps_max = 300

# Number of iteration stop criterion
it_max = 20



# The number of solutions that we will have in the solver after the first iteration
k_k = 10

In [32]:
#############################
### Main program ###
#############################

print("\n--------Main program is loading...---------")

# --------- Interaction with the solver
data = []
n, m, data, T_machine, T_duration, duration, optimalval = fm.get_data_from_file(file)

# --------- Call Solver constructor in Solver.py and create the variables of the model
model, solver, tasks = fm.initialize_solver(data, n, m, duration)

# ------------ Solve the model
print("\nSolving the model...")
msol, nb_solution, runtime = solver.solve(model, k, n, m, T_machine, optimalval, tps_max, it_max)

# ------------ Display the result
# fm.display_solution(msol, display_sol)
print("Model solved !")

# ---------------- Interaction with the user
print("\n--------Interaction with the user...---------")
print("\nCreating the user...")
user = User()
print("User created !")

#Get the variables of the model
variables = solver.get_variables()

list_indice, list_obj, pref, list_layers, list_equal = fm.user_preferences(msol, user, nb_layers, n, m)

# Vector of the start time of each task of each preference
starts = user.start_pref(n, m, variables, display_start)

# Matrix of the start time of each task of each preference
matrix = user.matrix_pref(n, m, display_matrix)

# Testing the order of preferences and the differences between solutions
fm.test(n, m, user)

print("list layers : ",list_layers)
print(list_layers)


--------Main program is loading...---------

Creating the model variables...
Model tasks_by_jobs created !

Solver initialized !

Solving the model...

Adding precedence constraints to the solver...
Precedence constraints added !

Adding disjunctive constraints to the solver...
Disjunctive constraints added !
Model solved !

--------Interaction with the user...---------

Creating the user...
User created !

Classing solutions...
Le min de list_obj est 2453016
Le max de list_obj est 3654828
Solutions classed !

Creating preferences...
Preferences created !

Testing order of preferences...
	L'ordre des préférences n'est pas cohérente

Testing differences between solutions...
	Toutes les solutions sont différentes
list layers :  [[<docplex.cp.solution.CpoSolveResult object at 0x000002199F8D9A40>, <docplex.cp.solution.CpoSolveResult object at 0x00000219A20BAEC0>, <docplex.cp.solution.CpoSolveResult object at 0x00000219ABDB9440>], [<docplex.cp.solution.CpoSolveResult object at 0x00000219A8

In [33]:
# a = 1 means that the order of the sol is 0 
# a = 0 means that the order of the sol is not 0

def activation_function(input, weights, nb_hidden_layers, nb_neurons, id_layer = 0):

    output = []
    for j in range(nb_neurons[id_layer]):
        sum = 0
        for k in range(len(input)):
            sum  += input[k] * weights[id_layer][j][k]
            
        if sum > 0: a = 1 
        else: a = 0

        if id_layer == nb_hidden_layers: return a
        else: output.append(a)

    return activation_function(output, weights, nb_hidden_layers, nb_neurons, id_layer + 1)
     
    

In [34]:
nb_hidden_layers1 = 2
nb_neurons1 = [3, 2, 1]

nb_hidden_layers2 = 2
nb_neurons2 = [3, 2, 1]

# ---------------- Interaction with the neural network

# Create the weights of the neural network
weights0 = [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(0,j,k)) for k in range(n*m)] for j in range(nb_neurons1[0])]]

weights1 = weights0 + [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(i,j,k)) for k in range(nb_neurons1[i-1])] for j in range(nb_neurons1[i])] for i in range(1, nb_hidden_layers1 + 1)]

# weights02 = [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(0,j,k)) for k in range(n*m)] for j in range(nb_neurons2[0])]]
weights2 = weights0 + [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(i,j,k)) for k in range(nb_neurons2[i-1])] for j in range(nb_neurons2[i])] for i in range(1, nb_hidden_layers2 + 1)]


# for i in range(nb_hidden_layers1 + 1):
#     weights1 = [[None for j in range(nb_neurons1[i])] for k in range(nb_neurons1[i-1])]
#     for j in range(nb_neurons1[i]):
#         if i == 0:
#             for k in range(n*m):
#                 print(i,j,k)
#                 print(weights1[i][j][k])
#                 weights1[i][j][k] = model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(i,j,k))
#         else:
#             for k in range(nb_neurons1[i-1]):
#                 weights1[i][j][k] = model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(i,j,k))


# Train the neural network

for order in range(len(list_layers)):
    for sol in list_layers[order]:

        output_NN1 = activation_function(sol, weights1, nb_hidden_layers1, nb_neurons1)
        output_NN2 = activation_function(sol, weights2, nb_hidden_layers2, nb_neurons2)

        if order == 0:
            solver.add_constraint(model, output_NN1 == 1)
            solver.add_constraint(model, output_NN2 == 1)
        else:
            solver.add_constraint(model, output_NN1 == 0)
            solver.add_constraint(model, output_NN2 == 0)


# Compare the result of 2 different neural networks for the same solution
        outputvar_NN1 = activation_function(variables, weights1, nb_hidden_layers1, nb_neurons1, nb_hidden_layers1)
        outputvar_NN2 = activation_function(variables, weights2, nb_hidden_layers2, nb_neurons2, nb_hidden_layers2)
        solver.add_constraint(model, outputvar_NN1 != outputvar_NN2)
        solver.add_constraint(model, outputvar_NN1 == 1 or outputvar_NN2 == 1)

TypeError: object of type 'CpoSolveResult' has no len()